# Data resizing before model training

In [ ]:
import os
import cv2
import time
import numpy as np
from tqdm import tqdm
import imgaug as ia
import imgaug.augmenters as iaa

## Define required custom functions

#### Create a coordinate format with labels
##### - category : class(disease name)



In [ ]:
def Yolo2Coord(size, box):
  category = int(box[0])
  x_center, y_center, width, height = float(box[1]), float(box[2]), float(box[3]), float(box[4])
  img_width, img_height = size[0], size[1]
  x_min = int((x_center - width/2.) * img_width)
  x_max = int((x_center + width/2.) * img_width)
  y_min = int((y_center - height/2.) * img_height)
  y_max = int((y_center + height/2.) * img_height)
  return [category, x_min, y_min, x_max, y_max]

##### Create a YOLO label format with coordinates (x_min, y_min, x_max, y_max)

In [ ]:
def Pascal2Yolo(box, size):
    image_width, image_height = size[0], size[1]
    x_center = ((box[2] + box[0]) / 2) / (image_width)
    y_center = ((box[3] + box[1]) / 2) / (image_height)
    width = (box[2] - box[0]) / (image_width)
    height = (box[3] - box[1]) / (image_height)
    return [str(x_center), str(y_center), str(width), str(height)]

##### coordinates after resizing


In [ ]:
def after_box(bbs):
  before = []
  for i in range(len(bbs)):
    x_min = bbs[i][0][0]
    y_min = bbs[i][0][1]
    x_max = bbs[i][1][0]
    y_max = bbs[i][1][1]
    before.append([x_min, y_min, x_max, y_max])
  return before

## Route Settings

In [ ]:
train_img_root_path = "/content/drive/MyDrive/crop_disease/train/images/"
train_label_root_path = "/content/drive/MyDrive/crop_disease/train/labels/"
val_img_root_path = "/content/drive/MyDrive/crop_disease/valid/images/"
val_label_root_path = "/content/drive/MyDrive/crop_disease/valid/labels/"

train_image_path = "/content/drive/MyDrive/crop_disease/train/imagess/"
train_label_path = "/content/drive/MyDrive/crop_disease/train/labelss/"

val_image_path = "/content/drive/MyDrive/crop_disease/valid/imagess/"
val_label_path = "/content/drive/MyDrive/crop_disease/valid/labelss/"

train_images = list(os.listdir(train_image_path))
train_labels = list(os.listdir(train_label_path))

val_images = list(os.listdir(val_image_path))
val_labels = list(os.listdir(val_label_path))

##### Set YOLOv5 model image size

In [ ]:
size = 640

## Resizing

In [ ]:
# train data
for img in tqdm(train_images):
  image = cv2.imread(train_image_path + img) # 이미지 불러오기
  label = img[:-4] + ".txt" # 라벨링 데이터 불러오기
  height, width = image.shape[:2] # 원천 이미지 픽셀 불러오기 (height, weight, dimension)
  image_size = [width, height] # cv2는 weight * height를 출력하기에 width, height 형식으로 변환

  # bounding box 라벨 저장
  label_list = [] # 텍스트 파일에 있는 라벨을 담을 list

  with open(train_label_path + label, "r") as f:
    labs = f.readlines() 
    for lab in labs:
      label_list.append(lab.split()) # save label in list
    f.close()

  """
  bounding box label은 category(클래스:병명), x_center, y_center, width, height 형식
  이를 좌표값 형식(category, x_min, y_min, x_max, y_max)로 변환
  Yolo2Coord() : x_center, y_center, width, height → x_min(좌상단 x), y_min(좌상단 y), x_max(우하단 x), y_max(우하단 y)
  """

  new_bounding_boxes = []
  for box in label_list:
    new_bounding_boxes.append(Yolo2Coord(image_size, box))
  cate = str(new_bounding_boxes[0][0]) # 밑에서 사용

  # augmentation 할 때 쓰는 BoundingBox 타입으로 변환
  ia_bounding_boxes = []

  for box in new_bounding_boxes:
    ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

  # resizing
  bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

  seq = iaa.Resize({"height":size, "width":size})


  image_resized, bbs_resized = seq(image = image, bounding_boxes = bbs) # bbs_aug : resized bounding box 좌표

  cv2.imwrite(train_img_root_path + "_resized_" +img, image_resized) # resized image 저장 경로

  # resized image coordinate
  resized = image_resized.shape # 좌표화 된 bounding box label을 YOLO 형식으로 변환할 때 필요
  rotated_size = [resized[1], resized[0]] # 마찬가지로 height, width -> width, height 형식으로 변환

  # after_box() : boundingbox 형식으로 묶여있는 x_min, y_min, x_max, y_max 좌표를 리스트로 정리
  resized_coords = after_box(bbs_resized)

  yolos = []
  for i in range(len(resized_coords)):
    a = Pascal2Yolo(resized_coords[i], rotated_size)
    final = cate + " " + " ".join(a)
    yolos.append(final)

  f = open(train_label_root_path + "_resized_" + label, "w", encoding = "utf-8", newline='') # resized label 저장 경로
  for yolo in yolos:
    f.write(yolo)
    f.write("\n")
  f.close()

100%|██████████| 17468/17468 [3:39:56<00:00,  1.32it/s]


In [ ]:
# valid data
for img in tqdm(val_images):
  image = cv2.imread(val_image_path + img)
  label = img[:-4] + ".txt"
  height, width = image.shape[:2]
  image_size = [width, height]

  label_list = []

  with open(val_label_path + label, "r") as f:
    labs = f.readlines() 
    for lab in labs:
      label_list.append(lab.split())
    f.close()

  new_bounding_boxes = []
  for box in label_list:
    new_bounding_boxes.append(Yolo2Coord(image_size, box))
  cate = str(new_bounding_boxes[0][0])

  ia_bounding_boxes = []

  for box in new_bounding_boxes:
    ia_bounding_boxes.append(ia.BoundingBox(x1 = box[1], y1 = box[2], x2 = box[3], y2 = box[4]))

  bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape = image.shape)

  seq = iaa.Resize({"height":size, "width":size})


  image_resized, bbs_resized = seq(image = image, bounding_boxes = bbs)

  cv2.imwrite(val_img_root_path + "_resized_" + img, image_resized)

  resized = image_resized.shape
  rotated_size = [resized[1], resized[0]]

  resized_coords = after_box(bbs_resized)

  yolos = []
  for i in range(len(resized_coords)):
    a = Pascal2Yolo(resized_coords[i], rotated_size)
    final = cate + " " + " ".join(a)
    yolos.append(final)

  f = open(val_label_root_path + "_resized_" + label, "w", encoding = "utf-8", newline='')
  for yolo in yolos:
    f.write(yolo)
    f.write("\n")
  f.close()

100%|██████████| 3783/3783 [46:59<00:00,  1.34it/s]
